# Cleaning Kworb Data

In [2]:
# Imports
import pandas as pd

In [6]:
# Load in kworb data
raw_kword_df = pd.read_csv('/Users/joseflemker/Documents/GitHub/3250-data-analytics-template/data/rawstream_dataRaw.csv')

In [7]:
# View data
raw_kword_df.head(5)

,Artist/Song,Streams,Daily
0,The Weeknd - Blinding Lights,"4,560,739,811","1,912,741"
1,Ed Sheeran - Shape of You,"4,096,124,219","1,615,655"
2,Lewis Capaldi - Someone You Loved,"3,671,556,765","1,475,856"
3,Harry Styles - As It Was,"3,631,721,146","1,533,219"
4,Post Malone - Sunflower,"3,595,013,796","1,461,811"


In [10]:
# Check for duplicates and null values
duplicates = raw_kword_df.duplicated()
print(f'Duplicate count: {duplicates.sum()}')

Duplicate count: 0


In [17]:
# Check for artist/song duplicates
artistSong_duplicates = raw_kword_df['Artist/Song'].duplicated()
print(f'Duplicate artist/song count: {artistSong_duplicates.sum()}')

Duplicate artist/song count: 25


In [19]:
# Examine duplicates
print(raw_kword_df[artistSong_duplicates])

                                     Artist/Song        Streams    Daily
227                 The Weeknd - Save Your Tears  1,687,728,759  874,623
414   White Noise Baby Sleep - Clean White Noise  1,361,773,739      NaN
420                         Ed Sheeran - Shivers  1,356,315,091      NaN
734                    Lil Nas X - Old Town Road  1,039,628,178  189,454
767                     The Weeknd - Die For You  1,024,078,581  731,164
802                        Dua Lipa - Levitating  1,004,917,448  322,553
831                          Lizzo - Truth Hurts    990,690,505  161,441
1100  Elvis Presley - Can't Help Falling in Love    846,573,905  497,676
1301                         Sia - Cheap Thrills    769,190,813  345,191
1303                 Olivia Rodrigo - All I Want    768,847,035  487,824
1321                  Jack Harlow - Whats Poppin    763,671,243  119,009
1437                          Taylor Swift - ME!    730,859,004  320,741
1673             Coldplay - Hymn for the Weekend   

In [15]:
# Check for null values
print(f'Null artists/songs: {raw_kword_df['Artist/Song'].isna().sum()}')
print(f'Null streams: {raw_kword_df['Streams'].isna().sum()}')
print(f'Null daily streams: {raw_kword_df['Daily'].isna().sum()}')

Null artists/songs: 0
Null streams: 0
Null daily streams: 4


In [13]:
# Examine Null daily streams
raw_kword_df[raw_kword_df['Daily'].isna()]

,Artist/Song,Streams,Daily
414,White Noise Baby Sleep - Clean White Noise,"1,361,773,739",NaN
420,Ed Sheeran - Shivers,"1,356,315,091",NaN
733,Brenda Lee - Rockin' Around The Christmas Tree,"1,040,733,345",NaN
1895,Lil Tjay - F.N,"624,709,072",NaN
